In [1]:
import pandas as pd
# Then we'll load in our CSV file
df = pd.read_csv('datasets/Admission_Predict.csv', index_col=0)
# And we'll clean up a couple of poorly named columns like we did in a previous lecture
df.columns = [x.lower().strip() for x in df.columns]
# And we'll take a look at the results
df.head()

,gre score,toefl score,university rating,sop,lor,cgpa,research,chance of admit
Serial No.,,,,,,,,
1,337,118,4,4.5,4.5,9.65,1,0.92
2,324,107,4,4.0,4.5,8.87,1,0.76
3,316,104,3,3.0,3.5,8.00,1,0.72
4,322,110,3,3.5,2.5,8.67,1,0.80
5,314,103,2,2.0,3.0,8.21,0,0.65


* querying dataframes is all about boolean masking

In [3]:
admit_mask=df['chance of admit'] > 0.7
admit_mask

Serial No.
1       True
2       True
3       True
4       True
5      False
       ...  
396     True
397     True
398     True
399    False
400     True
Name: chance of admit, Length: 400, dtype: bool

* we can apply a mask in a couple of ways

In [4]:
df[admit_mask].head()

,gre score,toefl score,university rating,sop,lor,cgpa,research,chance of admit
Serial No.,,,,,,,,
1,337,118,4,4.5,4.5,9.65,1,0.92
2,324,107,4,4.0,4.5,8.87,1,0.76
3,316,104,3,3.0,3.5,8.00,1,0.72
4,322,110,3,3.5,2.5,8.67,1,0.80
6,330,115,5,4.5,3.0,9.34,1,0.90


* of course, you don't have to make the mask object (and likely won't)

In [11]:
#df[df['chance of admit'] > 0.7].head()
#df.head()[df['chance of admit'].head() > 0.7]
df['chance of admit'].head() > 0.7

Serial No.
1     True
2     True
3     True
4     True
5    False
Name: chance of admit, dtype: bool

* we can also use the `where()` function, a subtle issue is that NaN's are left in for you.

In [12]:
df.where(admit_mask).head()

,gre score,toefl score,university rating,sop,lor,cgpa,research,chance of admit
Serial No.,,,,,,,,
1,337.0,118.0,4.0,4.5,4.5,9.65,1.0,0.92
2,324.0,107.0,4.0,4.0,4.5,8.87,1.0,0.76
3,316.0,104.0,3.0,3.0,3.5,8.00,1.0,0.72
4,322.0,110.0,3.0,3.5,2.5,8.67,1.0,0.80
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


* the nice thing about `where()` is that it's easy to read
* often you mix it together with `dropna()`

In [13]:
df.where(admit_mask).dropna().head()

,gre score,toefl score,university rating,sop,lor,cgpa,research,chance of admit
Serial No.,,,,,,,,
1,337.0,118.0,4.0,4.5,4.5,9.65,1.0,0.92
2,324.0,107.0,4.0,4.0,4.5,8.87,1.0,0.76
3,316.0,104.0,3.0,3.0,3.5,8.00,1.0,0.72
4,322.0,110.0,3.0,3.5,2.5,8.67,1.0,0.80
6,330.0,115.0,5.0,4.5,3.0,9.34,1.0,0.90


* masks can be composites, and made up of several conditions

In [14]:
(df['chance of admit'] > 0.7) and (df['chance of admit'] < 0.9)

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

* the problem is, pandas doesn't know how to `and` two `Series` objects together.
* PEP 335: https://www.python.org/dev/peps/pep-0335/

* But it does know how to `&` them!

In [15]:
(df['chance of admit'] > 0.7) & (df['chance of admit'] < 0.9)

Serial No.
1      False
2       True
3       True
4       True
5      False
       ...  
396     True
397     True
398    False
399    False
400    False
Name: chance of admit, Length: 400, dtype: bool

* But, you need to watch out for order of operations!

In [16]:
df[df['chance of admit'] > 0.7 & df['chance of admit'] < 0.9]

TypeError: cannot compare a dtyped [float64] array with a scalar of type [bool]

* finally, there are additional helped functions on dataframes to be aware of

In [17]:
df.where(df['chance of admit'].gt(0.7)).dropna().head()

,gre score,toefl score,university rating,sop,lor,cgpa,research,chance of admit
Serial No.,,,,,,,,
1,337.0,118.0,4.0,4.5,4.5,9.65,1.0,0.92
2,324.0,107.0,4.0,4.0,4.5,8.87,1.0,0.76
3,316.0,104.0,3.0,3.0,3.5,8.00,1.0,0.72
4,322.0,110.0,3.0,3.5,2.5,8.67,1.0,0.80
6,330.0,115.0,5.0,4.5,3.0,9.34,1.0,0.90


# Indexing
* Let's go back to indexing dataframes, there's some neat stuff there
* Remember that the index are row level labels, and the column names are the column level labels
* We can swap columns and rows trivially

In [18]:
df.T

Serial No.,1,2,3,4,5,6,7,8,9,10,...,391,392,393,394,395,396,397,398,399,400
gre score,337.00,324.00,316.00,322.00,314.00,330.00,321.00,308.00,302.0,323.00,...,314.00,318.00,326.00,317.00,329.00,324.00,325.00,330.00,312.00,333.00
toefl score,118.00,107.00,104.00,110.00,103.00,115.00,109.00,101.00,102.0,108.00,...,102.00,106.00,112.00,104.00,111.00,110.00,107.00,116.00,103.00,117.00
university rating,4.00,4.00,3.00,3.00,2.00,5.00,3.00,2.00,1.0,3.00,...,2.00,3.00,4.00,2.00,4.00,3.00,3.00,4.00,3.00,4.00
sop,4.50,4.00,3.00,3.50,2.00,4.50,3.00,3.00,2.0,3.50,...,2.00,2.00,4.00,3.00,4.50,3.50,3.00,5.00,3.50,5.00
lor,4.50,4.50,3.50,2.50,3.00,3.00,4.00,4.00,1.5,3.00,...,2.50,3.00,3.50,3.00,4.00,3.50,3.50,4.50,4.00,4.00
cgpa,9.65,8.87,8.00,8.67,8.21,9.34,8.20,7.90,8.0,8.60,...,8.24,8.65,9.12,8.76,9.23,9.04,9.11,9.45,8.78,9.66
research,1.00,1.00,1.00,1.00,0.00,1.00,1.00,0.00,0.0,0.00,...,0.00,0.00,1.00,0.00,1.00,1.00,1.00,1.00,0.00,1.00
chance of admit,0.92,0.76,0.72,0.80,0.65,0.90,0.75,0.68,0.5,0.45,...,0.64,0.71,0.84,0.77,0.89,0.82,0.84,0.91,0.67,0.95


* we saw that we can set the index with `set_index()`

In [19]:
df.set_index('lor').head()

,gre score,toefl score,university rating,sop,cgpa,research,chance of admit
lor,,,,,,,
4.5,337,118,4,4.5,9.65,1,0.92
4.5,324,107,4,4.0,8.87,1,0.76
3.5,316,104,3,3.0,8.00,1,0.72
2.5,322,110,3,3.5,8.67,1,0.80
3.0,314,103,2,2.0,8.21,0,0.65


In [23]:
# of course, this didn't actually change our previous dataframe, right?
display(df.head())
ndf=df.where(df["sop"]>4.1)
ndf.head()

,gre score,toefl score,university rating,sop,lor,cgpa,research,chance of admit
Serial No.,,,,,,,,
1,337,118,4,4.5,4.5,9.65,1,0.92
2,324,107,4,4.0,4.5,8.87,1,0.76
3,316,104,3,3.0,3.5,8.00,1,0.72
4,322,110,3,3.5,2.5,8.67,1,0.80
5,314,103,2,2.0,3.0,8.21,0,0.65


,gre score,toefl score,university rating,sop,lor,cgpa,research,chance of admit
Serial No.,,,,,,,,
1,337.0,118.0,4.0,4.5,4.5,9.65,1.0,0.92
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Multilevel indexing
* we can have heirarchical indicies, which is pretty cool
* let's look at some census data

In [24]:
import pandas as pd 
df=pd.read_csv("datasets/census.csv")
df.head()

,SUMLEV,REGION,DIVISION,STATE,COUNTY,STNAME,CTYNAME,CENSUS2010POP,ESTIMATESBASE2010,POPESTIMATE2010,...,RDOMESTICMIG2011,RDOMESTICMIG2012,RDOMESTICMIG2013,RDOMESTICMIG2014,RDOMESTICMIG2015,RNETMIG2011,RNETMIG2012,RNETMIG2013,RNETMIG2014,RNETMIG2015
0,40,3,6,1,0,Alabama,Alabama,4779736,4780127,4785161,...,0.002295,-0.193196,0.381066,0.582002,-0.467369,1.030015,0.826644,1.383282,1.724718,0.712594
1,50,3,6,1,1,Alabama,Autauga County,54571,54571,54660,...,7.242091,-2.915927,-3.012349,2.265971,-2.530799,7.606016,-2.626146,-2.722002,2.592270,-2.187333
2,50,3,6,1,3,Alabama,Baldwin County,182265,182265,183193,...,14.832960,17.647293,21.845705,19.243287,17.197872,15.844176,18.559627,22.727626,20.317142,18.293499
3,50,3,6,1,5,Alabama,Barbour County,27457,27457,27341,...,-4.728132,-2.500690,-7.056824,-3.904217,-10.543299,-4.874741,-2.758113,-7.167664,-3.978583,-10.543299
4,50,3,6,1,7,Alabama,Bibb County,22915,22919,22861,...,-5.527043,-5.068871,-6.201001,-0.177537,0.177258,-5.088389,-4.363636,-5.403729,0.754533,1.107861


In [28]:
# in this data there are only two sumlevels
#df["SUMLEV"].unique()
#print(len(df['SUMLEV'].unique()))
# so lets just get rid of anything except state level data
df=df[df['SUMLEV'] == 50]
df.head()

,SUMLEV,REGION,DIVISION,STATE,COUNTY,STNAME,CTYNAME,CENSUS2010POP,ESTIMATESBASE2010,POPESTIMATE2010,...,RDOMESTICMIG2011,RDOMESTICMIG2012,RDOMESTICMIG2013,RDOMESTICMIG2014,RDOMESTICMIG2015,RNETMIG2011,RNETMIG2012,RNETMIG2013,RNETMIG2014,RNETMIG2015
1,50,3,6,1,1,Alabama,Autauga County,54571,54571,54660,...,7.242091,-2.915927,-3.012349,2.265971,-2.530799,7.606016,-2.626146,-2.722002,2.592270,-2.187333
2,50,3,6,1,3,Alabama,Baldwin County,182265,182265,183193,...,14.832960,17.647293,21.845705,19.243287,17.197872,15.844176,18.559627,22.727626,20.317142,18.293499
3,50,3,6,1,5,Alabama,Barbour County,27457,27457,27341,...,-4.728132,-2.500690,-7.056824,-3.904217,-10.543299,-4.874741,-2.758113,-7.167664,-3.978583,-10.543299
4,50,3,6,1,7,Alabama,Bibb County,22915,22919,22861,...,-5.527043,-5.068871,-6.201001,-0.177537,0.177258,-5.088389,-4.363636,-5.403729,0.754533,1.107861
5,50,3,6,1,9,Alabama,Blount County,57322,57322,57373,...,1.807375,-1.177622,-1.748766,-2.062535,-1.369970,1.859511,-0.848580,-1.402476,-1.577232,-0.884411


In [31]:
# we can set a multilevel index just by passing a list of things we want to index on
#df=df.set_index(['STNAME', 'CTYNAME'])
df.head(20)

SUMLEV  REGION  DIVISION  STATE  COUNTY  \
STNAME  CTYNAME                                                     
Alabama Autauga County        50       3         6      1       1   
        Baldwin County        50       3         6      1       3   
        Barbour County        50       3         6      1       5   
        Bibb County           50       3         6      1       7   
        Blount County         50       3         6      1       9   
        Bullock County        50       3         6      1      11   
        Butler County         50       3         6      1      13   
        Calhoun County        50       3         6      1      15   
        Chambers County       50       3         6      1      17   
        Cherokee County       50       3         6      1      19   
        Chilton County        50       3         6      1      21   
        Choctaw County        50       3         6      1      23   
        Clarke County         50       3         6      1      25   
        Clay County           50       3         6      1      27   
        Cleburne County       50       3         6      1      29   
        Coffee County         50       3         6      1      31   
        Colbert County        50       3         6      1      33   
        Conecuh County        50       3         6      1      35   
        Coosa County          50       3         6      1      37   
        Covington County      50       3         6      1      39   

                          CENSUS2010POP  ESTIMATESBASE2010  POPESTIMATE2010  \
STNAME  CTYNAME                                                               
Alabama Autauga County            54571              54571            54660   
        Baldwin County           182265             182265           183193   
        Barbour County            27457              27457            27341   
        Bibb County               22915              22919            22861   
        Blount County             57322              57322            57373   
        Bullock County            10914              10915            10887   
        Butler County             20947              20946            20944   
        Calhoun County           118572             118586           118437   
        Chambers County           34215              34170            34098   
        Cherokee County           25989              25986            25976   
        Chilton County            43643              43631            43665   
        Choctaw County            13859              13858            13841   
        Clarke County             25833              25840            25767   
        Clay County               13932              13932            13880   
        Cleburne County           14972              14972            14973   
        Coffee County             49948              49948            50177   
        Colbert County            54428              54428            54514   
        Conecuh County            13228              13228            13208   
        Coosa County              11539              11758            11758   
        Covington County          37765              37765            37796   

                          POPESTIMATE2011  POPESTIMATE2012  ...  \
STNAME  CTYNAME                                             ...   
Alabama Autauga County              55253            55175  ...   
        Baldwin County             186659           190396  ...   
        Barbour County              27226            27159  ...   
        Bibb County                 22733            22642  ...   
        Blount County               57711            57776  ...   
        Bullock County              10629            10606  ...   
        Butler County               20673            20408  ...   
        Calhoun County             117768           117286  ...   
        Chambers County             33993            34075  ...   
        Cherokee County        

* Querying gets, frankly, complex
* Someone remind me, how does df.loc work again?
  * df.loc[X,Y]

* `df.loc[row, column]`
* But with a multiindex we can do
  * `df.loc[row index1, row index2]`

In [35]:
df.loc['Michigan', 'Washtenaw County']['REGION']

2.0

In [36]:
df.loc['Michigan', 'REGION']['Washtenaw County']

2

In [39]:
# It's a bit of ambiguous, I recommend passing keys as tuple instead
df.loc[('Michigan', 'Washtenaw County')]

SUMLEV  REGION  DIVISION  STATE  COUNTY  \
STNAME  CTYNAME                                                      
Alabama Autauga County         50       3         6      1       1   
        Baldwin County         50       3         6      1       3   
        Barbour County         50       3         6      1       5   
        Bibb County            50       3         6      1       7   
        Blount County          50       3         6      1       9   
...                           ...     ...       ...    ...     ...   
Wyoming Sweetwater County      50       4         8     56      37   
        Teton County           50       4         8     56      39   
        Uinta County           50       4         8     56      41   
        Washakie County        50       4         8     56      43   
        Weston County          50       4         8     56      45   

                           CENSUS2010POP  ESTIMATESBASE2010  POPESTIMATE2010  \
STNAME  CTYNAME                                                                
Alabama Autauga County             54571              54571            54660   
        Baldwin County            182265             182265           183193   
        Barbour County             27457              27457            27341   
        Bibb County                22915              22919            22861   
        Blount County              57322              57322            57373   
...                                  ...                ...              ...   
Wyoming Sweetwater County          43806              43806            43593   
        Teton County               21294              21294            21297   
        Uinta County               21118              21118            21102   
        Washakie County             8533               8533             8545   
        Weston County               7208               7208             7181   

                           POPESTIMATE2011  POPESTIMATE2012  ...  \
STNAME  CTYNAME                                              ...   
Alabama Autauga County               55253            55175  ...   
        Baldwin County              186659           190396  ...   
        Barbour County               27226            27159  ...   
        Bibb County                  22733            22642  ...   
        Blount County                57711            57776  ...   
...                                    ...              ...  ...   
Wyoming Sweetwater County            44041            45104  ...   
        Teton County                 21482            21697  ...   
        Uinta County                 20912            20989  ...   
        Washakie County               8469             8443  ...   
        Weston County                 7114             7065  ...   

                           RDOMESTICMIG2011  RDOMESTICMIG2012  \
STNAME  CTYNAME                                                 
Alabama Autauga County             7.242091         -2.915927   
        Baldwin County            14.832960         17.647293   
        Barbour County            -4.728132         -2.500690   
        Bibb County               -5.527043         -5.068871   
        Blount County              1.807375         -1.177622   
...                                     ...               ...   
Wyoming Sweetwater County          1.072643         16.243199   
        Teton County              -1.589565          0.972695   
        Uinta County             -17.755986         -4.916350   
        Washakie County          -11.637475         -0.827815   
        Weston County            -11.752361         -8.040059   

                           RDOMESTICMIG2013  RDOMESTICMIG2014  \
STNAME  CTYNAME                                                 
Alabama Autauga County            -3.012349          2.265971   
        Baldwin County            21.845705         19.243287   
        Barbour County            -7.056824         -3.904217   
        Bibb County

# In class activity time!
## Question 0: Which county has the largest population in Michigan?
* (Easy)

## Question 1: Generate a new column for which is the largest absolute change in population within the period 2010-2015?
* (Hint: population values are stored in columns POPESTIMATE2010 through POPESTIMATE2015, you need to consider all six columns.)

In [52]:
df.loc[('Michigan')].sort_values('CENSUS2010POP', ascending = False).head(1) 

,SUMLEV,REGION,DIVISION,STATE,COUNTY,CENSUS2010POP,ESTIMATESBASE2010,POPESTIMATE2010,POPESTIMATE2011,POPESTIMATE2012,...,RDOMESTICMIG2011,RDOMESTICMIG2012,RDOMESTICMIG2013,RDOMESTICMIG2014,RDOMESTICMIG2015,RNETMIG2011,RNETMIG2012,RNETMIG2013,RNETMIG2014,RNETMIG2015
CTYNAME,,,,,,,,,,,,,,,,,,,,,
Wayne County,50,2,3,26,163,1820584,1820641,1815199,1801273,1792514,...,-13.340073,-10.271616,-14.119617,-11.903253,-8.762835,-11.344758,-8.098421,-11.732437,-9.161648,-6.010195


In [51]:
ndf=df.loc["Michigan"]["POPESTIMATE2012"]
ndf[ndf==ndf.max()]

CTYNAME
Wayne County    1792514
Name: POPESTIMATE2012, dtype: int64

In [54]:
import numpy as np
df[["POPESTIMATE2012","POPESTIMATE2011","POPESTIMATE2013"]].apply(np.min)

POPESTIMATE2012    81
POPESTIMATE2011    90
POPESTIMATE2013    89
dtype: int64